# Pytorch Tutorial

Pytorch is a popular deep learning framework and it's easy to get started.

In [2]:
import torch
import torch.nn as nn
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
import time

BATCH_SIZE = 128
NUM_EPOCHS = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")

First, we read the mnist data, preprocess them and encapsulate them into dataloader form.

In [3]:
# preprocessing
normalize = transforms.Normalize(mean=[.5], std=[.5])
transform = transforms.Compose([transforms.ToTensor(), normalize])

# download and load the data
train_dataset = torchvision.datasets.MNIST(root='./mnist/', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./mnist/', train=False, transform=transform, download=False)

# encapsulate them into dataloader form
train_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
test_loader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)


Then, we define the model, object function and optimizer that we use to classify.

In [4]:
class SimpleNet(nn.Module):

    def __init__(self):
        super(SimpleNet,self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(1,10,5,1,1),
            nn.MaxPool2d(2),
            nn.ReLU(),
            nn.BatchNorm2d(10)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(10,20,5,1,1),
            nn.MaxPool2d(2),
            nn.ReLU(),
            nn.BatchNorm2d(20) 
        )
        self.fc1 = nn.Sequential(
            nn.Linear(500,60),
            nn.Dropout(0.5),
            nn.ReLU()
        )
        self.fc2 = nn.Sequential(
            nn.Linear(60,20),
            nn.Dropout(0.5),
            nn.ReLU()
        )
        self.fc3 = nn.Linear(20,10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(-1,500)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return x

    
model = SimpleNet().to(device)


criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(),lr=0.001,weight_decay=0.0)

Next, we can start to train and evaluate!

In [5]:
# train and evaluate
for epoch in range(NUM_EPOCHS):
    # train
    model.train()    
    train_loss = 0
    correct = 0
    for data, target in tqdm(train_loader):
        data = data.to(device) 
        target = target.to(device)
        output = model(data)
        # sum up batch loss
        loss = criterion(output, target).mean()
        # get the index of the max log-probability
        pred = output.data.max(1, keepdim=True)[1]
        # print(pred)
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

        optimizer.zero_grad()   
        loss.backward()         
        optimizer.step()  


    # train accuracy
    model.eval()
    train_loss = 0
    train_correct = 0
    for data, target in tqdm(train_loader):
        data = data.to(device) 
        target = target.to(device)
        output = model(data)
        # sum up batch loss
        loss = criterion(output, target).mean()
        train_loss += loss
        # get the index of the max log-probability
        pred = output.data.max(1, keepdim=True)[1]
        # print(pred)
        train_correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    train_loss /= len(train_loader.dataset)

    # test accuracy
    test_loss = 0
    test_correct = 0
    model.eval()
    for data, target in tqdm(test_loader):
        data = data.to(device) 
        target = target.to(device)
        output = model(data)
        # sum up batch loss
        test_loss += criterion(output, target).mean()
        # get the index of the max log-probability
        pred = output.data.max(1, keepdim=True)[1]
        # print(pred)
        test_correct += pred.eq(target.data.view_as(pred)).cpu().sum()


    test_loss /= len(test_loader.dataset)


    print('epoch {}'.format(epoch))
    print('Train set: Average loss: {:.4f}, Accuracy: {}/{} ({:.3f}%)'.format(
        train_loss, train_correct, len(train_loader.dataset),
        100 * float(train_correct) / len(train_loader.dataset)))
    print('Test  set: Average loss: {:.4f}, Accuracy: {}/{} ({:.3f}%)'.format(
        test_loss, test_correct, len(test_loader.dataset),
        100 * float(test_correct) / len(test_loader.dataset)))

    torch.cuda.empty_cache()

100%|██████████| 78/78 [00:11<00:00,  7.01it/s]
epoch 0
Train set: Average loss: 0.0006, Accuracy: 58578/60000 (97.630%)
Test  set: Average loss: 0.0006, Accuracy: 9767/10000 (97.670%)
100%|██████████| 78/78 [00:08<00:00,  9.65it/s]
epoch 1
Train set: Average loss: 0.0004, Accuracy: 59034/60000 (98.390%)
Test  set: Average loss: 0.0005, Accuracy: 9814/10000 (98.140%)
100%|██████████| 78/78 [00:10<00:00,  7.34it/s]
epoch 2
Train set: Average loss: 0.0004, Accuracy: 59154/60000 (98.590%)
Test  set: Average loss: 0.0004, Accuracy: 9846/10000 (98.460%)
100%|██████████| 78/78 [00:08<00:00,  9.65it/s]
epoch 3
Train set: Average loss: 0.0003, Accuracy: 59196/60000 (98.660%)
Test  set: Average loss: 0.0004, Accuracy: 9845/10000 (98.450%)
100%|██████████| 78/78 [00:08<00:00,  8.88it/s]
epoch 4
Train set: Average loss: 0.0002, Accuracy: 59448/60000 (99.080%)
Test  set: Average loss: 0.0003, Accuracy: 9871/10000 (98.710%)
100%|██████████| 78/78 [00:09<00:00,  7.88it/s]
epoch 5
Train set: Average 

#### Q5:
Please print the training and testing accuracy.